# Bedrock 지식 베이스와 액션 그룹이 통합된 에이전트 만들기

이 노트북에서는 Amazon Bedrock의 지식 베이스(Knowledge Bases)를 활용하여 레스토랑 메뉴 정보를 검색하는 Amazon Bedrock 에이전트를 만드는 방법을 배우게 됩니다. 

**실습 시나리오:** 레스토랑 AI 도우미 만들기
- 성인 메뉴와 어린이 메뉴에 대한 정보 제공
- 테이블 예약 시스템 관리 (예약 생성, 삭제, 조회)

<img src="./images/architecture.png" style="width:70%;display:block;margin: 0 auto;">
<br/>

**실습 진행 단계:**

1. 필요한 라이브러리 가져오기
2. Amazon Bedrock용 지식 베이스 생성
3. 데이터셋을 Amazon S3에 업로드
4. Amazon Bedrock 에이전트 생성
5. 에이전트 테스트
6. 생성된 리소스 정리

## 1. 필요한 라이브러리 가져오기

먼저 필요한 패키지들을 설치합니다.

In [ ]:
!pip install --upgrade -q -r requirements.txt

In [ ]:
import os
import time
import boto3
import logging
import pprint
import json

# 사용자 정의 모듈들 (이미 준비된 헬퍼 클래스들)
from knowledge_base import BedrockKnowledgeBase  # 지식 베이스 관리용
from agent import create_agent_role_and_policies, create_lambda_role, delete_agent_roles_and_policies  # 에이전트 및 권한 관리용
from agent import create_dynamodb, create_lambda, clean_up_resources  # 리소스 생성 및 정리용

In [ ]:
# AWS 클라이언트 설정
# 각 AWS 서비스에 접근하기 위한 클라이언트들을 초기화합니다
s3_client = boto3.client('s3')  # S3 파일 저장소 클라이언트
sts_client = boto3.client('sts')  # AWS 계정 정보 확인용 클라이언트
session = boto3.session.Session()
region = session.region_name  # 현재 AWS 리전
account_id = sts_client.get_caller_identity()["Account"]  # AWS 계정 ID
bedrock_agent_client = boto3.client('bedrock-agent')  # Bedrock 에이전트 관리 클라이언트
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')  # 에이전트 실행용 클라이언트

# 로그 설정
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

print(f"현재 리전: {region}")
print(f"AWS 계정 ID: {account_id}")
region, account_id

In [ ]:
# 설정 변수들 정의
# 이 변수들은 실습에서 생성할 리소스들의 이름을 정의합니다
suffix = f"{region}-{account_id}"  # 리소스 이름을 고유하게 만들기 위한 접미사
agent_name = 'booking-agent'  # 에이전트 이름
knowledge_base_name = f'{agent_name}-kb'  # 지식 베이스 이름
knowledge_base_description = "레스토랑 메뉴 정보를 담고 있는 지식 베이스"  # 지식 베이스 설명
agent_alias_name = "booking-agent-alias"  # 에이전트 별칭
bucket_name = f'{agent_name}-{suffix}'  # S3 버킷 이름
agent_bedrock_allow_policy_name = f"{agent_name}-ba"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'  # IAM 역할 이름
agent_description = "레스토랑 테이블 예약을 담당하는 에이전트"  # 에이전트 설명

# 에이전트에게 제공할 지시사항
agent_instruction = """
당신은 레스토랑 예약 도우미 에이전트입니다. 고객의 예약 정보 조회, 
새로운 예약 생성, 기존 예약 삭제를 도와주는 역할을 합니다.
"""

# 액션 그룹 설명 (에이전트가 수행할 수 있는 작업들)
agent_action_group_description = """
테이블 예약 정보 조회, 새 예약 생성, 기존 예약 삭제를 위한 액션들"""

agent_action_group_name = "TableBookingsActionGroup"  # 액션 그룹 이름

print(f"에이전트 이름: {agent_name}")
print(f"지식 베이스 이름: {knowledge_base_name}")
print(f"S3 버킷 이름: {bucket_name}")

**사용할 AI 모델 선택**

에이전트가 사용할 기본 AI 모델을 지정합니다. 아래 모델 중 하나를 선택할 수 있습니다:
- **Amazon Nova Pro**: us.amazon.nova-pro-v1:0 (Amazon의 최신 언어 모델)
- **Anthropic Claude 3.5 Sonnet v2**: us.anthropic.claude-3-sonnet-20240229-v1:0 (강력한 추론 능력)

In [ ]:
# 사용할 모델 선택 (필요에 따라 주석을 바꿔서 다른 모델 사용 가능)
# inference_profile = "us.anthropic.claude-3-sonnet-20240229-v1:0"  # Claude 모델
inference_profile = "us.amazon.nova-pro-v1:0"  # Nova Pro 모델 (기본 선택)
foundation_model = inference_profile[3:]  # 모델 ID에서 앞의 'us.' 제거

print(f"선택된 모델: {foundation_model}")

## 2. Amazon Bedrock용 지식 베이스 생성하기

**지식 베이스란?**
[Amazon Bedrock 지식 베이스](https://aws.amazon.com/bedrock/knowledge-bases/)는 레스토랑 메뉴 정보를 저장하고 검색할 수 있는 AI 검색 시스템입니다.

**지원하는 벡터 데이터베이스:**
- [Amazon OpenSearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/) (이번 실습에서 사용)
- [Amazon Aurora](https://aws.amazon.com/rds/aurora/)
- [Pinecone](http://app.pinecone.io/bedrock-integration)

**`BedrockKnowledgeBase` 헬퍼 클래스가 자동으로 생성하는 것들:**
1. **IAM 역할 및 정책** - 권한 관리
2. **S3 버킷** - 데이터 저장소
3. **OpenSearch Serverless 정책들** - 암호화, 네트워크, 데이터 접근 권한
4. **OpenSearch Serverless 컬렉션** - 검색 엔진
5. **벡터 인덱스** - 의미 검색을 위한 색인
6. **지식 베이스 본체**
7. **데이터 소스 연결**

In [ ]:
# 지식 베이스 생성
# 이 과정은 몇 분 정도 소요될 수 있습니다
print("지식 베이스를 생성하고 있습니다... 잠시만 기다려주세요.")

knowledge_base = BedrockKnowledgeBase(
    kb_name=knowledge_base_name,  # 지식 베이스 이름
    kb_description=knowledge_base_description,  # 지식 베이스 설명
    data_bucket_name=bucket_name  # 데이터가 저장될 S3 버킷 이름
)

print("지식 베이스 생성이 완료되었습니다!")

## 3. 데이터셋을 Amazon S3에 업로드하기

**데이터 업로드 과정**
이제 지식 베이스에 레스토랑 메뉴 정보를 넣어보겠습니다. 지식 베이스는 S3 버킷에 저장된 데이터를 읽어와서 검색 가능한 형태로 변환합니다. 

**동기화 과정:**
1. `dataset` 폴더의 메뉴 파일들을 S3에 업로드
2. `StartIngestionJob` API를 사용해서 지식 베이스와 동기화

In [ ]:
def upload_directory(path, bucket_name):
    """디렉토리의 모든 파일을 S3 버킷에 업로드하는 함수"""
    print(f"{path} 폴더의 파일들을 {bucket_name} 버킷에 업로드 중...")
    for root, dirs, files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root, file)
            print(f"  {file_to_upload} 업로드 중...")
            s3_client.upload_file(file_to_upload, bucket_name, file)
    print("모든 파일 업로드 완료!")

# dataset 폴더의 모든 파일을 S3에 업로드
upload_directory("dataset", bucket_name)

**이제 지식 베이스 동기화 작업을 시작합니다**

업로드된 파일들을 지식 베이스가 읽고 검색 가능한 형태로 변환하는 과정입니다.

In [ ]:
# 지식 베이스가 완전히 준비될 때까지 잠시 대기
print("지식 베이스 준비 중... (30초 대기)")
time.sleep(30)

# 지식 베이스 동기화 시작
print("지식 베이스 동기화 작업을 시작합니다...")
knowledge_base.start_ingestion_job()
print("동기화 작업이 완료되었습니다!")

**지식 베이스 ID 가져오기**

나중에 에이전트와 연결할 때 사용할 지식 베이스 ID를 저장합니다.

In [ ]:
kb_id = knowledge_base.get_knowledge_base_id()
print(f"지식 베이스 ID: {kb_id}")

### 3.1 지식 베이스 테스트해보기

**지식 베이스 동작 확인**
이제 지식 베이스가 제대로 작동하는지 테스트해보겠습니다. Amazon Bedrock은 두 가지 방법을 제공합니다:

1. **[retrieve](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html)** - 관련 정보만 검색
2. **[retrieve_and_generate](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html)** - 검색 + AI 답변 생성

#### 방법 1: Retrieve and Generate API로 테스트

이 방법은 지식 베이스에서 관련 정보를 찾고, AI 모델이 자동으로 최종 답변을 생성해줍니다.

In [ ]:
# 지식 베이스에 질문하기 (검색 + 답변 생성)
print("질문: 어린이 메뉴에는 어떤 메인 요리 5가지가 있나요?")
print("지식 베이스에서 답변을 찾는 중...")

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": "어린이 메뉴에는 어떤 메인 요리 5가지가 있나요?"
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,  # 우리가 만든 지식 베이스 ID
            "modelArn": f"arn:aws:bedrock:{region}:{account_id}:inference-profile/{inference_profile}",  # 사용할 AI 모델
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults": 5  # 검색할 결과 개수
                } 
            }
        }
    }
)

print("\nAI 답변:")
print("-" * 50)
print(response['output']['text'])
print("-" * 50)

**방법 1의 특징:**
위의 방법은 최종 답변만 보여주고, 어떤 자료를 참고했는지는 보여주지 않습니다.

#### 방법 2: Retrieve API로 상세 정보 확인

더 세밀한 제어가 필요하다면 retrieve API를 사용할 수 있습니다. 이 방법은 검색된 원본 텍스트 조각들, 파일 위치, 유사도 점수, 메타데이터 등을 모두 확인할 수 있습니다.

In [ ]:
# 지식 베이스에서 상세 정보 검색
print("같은 질문으로 상세 검색 결과를 확인해보겠습니다:")

response_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults": 5,  # 검색할 결과 개수
        } 
    },
    retrievalQuery={
        'text': '어린이 메뉴에는 어떤 메인 요리 5가지가 있나요?'
    }
)

def response_print(retrieve_resp):
    """검색 결과를 보기 좋게 출력하는 함수"""
    print(f"\n총 {len(retrieve_resp['retrievalResults'])}개의 관련 정보를 찾았습니다:\n")
    
    for num, chunk in enumerate(retrieve_resp['retrievalResults'], 1):
        print(f"정보 조각 {num}:")
        print(f"내용: {chunk['content']['text']}")
        print(f"파일 위치: {chunk['location']}")
        print(f"유사도 점수: {chunk['score']:.4f}")
        print(f"메타데이터: {chunk['metadata']}")
        print("-" * 80)

response_print(response_ret)

## 4. Amazon Bedrock 에이전트 만들기

**이제 본격적으로 에이전트를 만들어보겠습니다!**

에이전트는 지식 베이스(메뉴 정보 검색)와 액션 그룹(예약 관리)을 모두 사용할 수 있는 똑똑한 AI 도우미입니다.

**에이전트 생성 단계:**
    
1. **DynamoDB 테이블 생성** - 예약 정보 저장용 데이터베이스
2. **AWS Lambda 함수 생성** - 예약 관련 작업을 수행하는 코드
3. **IAM 권한 설정** - 에이전트가 필요한 권한들 부여
4. **에이전트 본체 생성** 
5. **액션 그룹 추가** - 에이전트가 할 수 있는 작업들 정의
6. **Lambda 호출 권한 설정**
7. **지식 베이스 연결**
8. **에이전트 준비 및 별칭 생성**

### 4.1 DynamoDB 테이블 생성

**예약 정보 저장소 만들기**
레스토랑 예약 정보를 저장할 DynamoDB 테이블을 생성합니다. DynamoDB는 AWS의 NoSQL 데이터베이스 서비스입니다.

In [ ]:
table_name = 'restaurant_bookings'  # 테이블 이름
print(f"DynamoDB 테이블 '{table_name}' 생성 중...")
create_dynamodb(table_name)
print("DynamoDB 테이블이 성공적으로 생성되었습니다!")

### 4.2 Lambda 함수 생성

**예약 관리 기능 구현하기**
에이전트가 예약 작업을 수행할 수 있도록 Lambda 함수를 만들겠습니다. Lambda는 서버 없이 코드를 실행할 수 있는 AWS 서비스입니다.

**진행 과정:**
1. **`lambda_function.py` 파일 작성** - 예약 관리 로직 구현
2. **IAM 역할 생성** - Lambda가 DynamoDB에 접근할 수 있는 권한 부여
3. **Lambda 함수 생성 및 배포**

#### 함수 코드 작성

**Lambda 함수 구현**
Amazon Bedrock 에이전트는 액션 그룹을 통해 Lambda 함수를 호출할 수 있습니다. 이 함수는 예약 조회(`get_booking_details`), 예약 생성(`create_booking`), 예약 삭제(`delete_booking`) 기능을 제공합니다.

In [ ]:
%%writefile lambda_function.py
import json
import uuid
import boto3

# DynamoDB 리소스 연결
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('restaurant_bookings')

def get_named_parameter(event, name):
    """
    Lambda 이벤트에서 특정 이름의 매개변수 값을 가져오는 함수
    
    Args:
        event: Lambda 이벤트 객체
        name: 찾을 매개변수 이름
    
    Returns:
        매개변수 값
    """
    return next(item for item in event['parameters'] if item['name'] == name)['value']


def get_booking_details(booking_id):
    """
    예약 정보를 조회하는 함수
    
    Args:
        booking_id (string): 조회할 예약 ID
    
    Returns:
        예약 정보 또는 오류 메시지
    """
    try:
        response = table.get_item(Key={'booking_id': booking_id})
        if 'Item' in response:
            return response['Item']
        else:
            return {'message': f'예약 ID {booking_id}에 해당하는 예약을 찾을 수 없습니다.'}
    except Exception as e:
        return {'error': str(e)}


def create_booking(date, name, hour, num_guests):
    """
    새로운 예약을 생성하는 함수
    
    Args:
        date (string): 예약 날짜
        name (string): 예약자 이름
        hour (string): 예약 시간
        num_guests (integer): 예약 인원 수
    
    Returns:
        생성된 예약 ID 또는 오류 메시지
    """
    try:
        booking_id = str(uuid.uuid4())[:8]  # 8자리 고유 ID 생성
        table.put_item(
            Item={
                'booking_id': booking_id,
                'date': date,
                'name': name,
                'hour': hour,
                'num_guests': num_guests
            }
        )
        return {'booking_id': booking_id, 'message': '예약이 성공적으로 생성되었습니다.'}
    except Exception as e:
        return {'error': str(e)}


def delete_booking(booking_id):
    """
    기존 예약을 삭제하는 함수
    
    Args:
        booking_id (str): 삭제할 예약 ID
    
    Returns:
        삭제 결과 메시지
    """
    try:
        response = table.delete_item(Key={'booking_id': booking_id})
        if response['ResponseMetadata']['HTTPStatusCode'] == 200:
            return {'message': f'예약 ID {booking_id}가 성공적으로 삭제되었습니다.'}
        else:
            return {'message': f'예약 ID {booking_id} 삭제에 실패했습니다.'}
    except Exception as e:
        return {'error': str(e)}
    

def lambda_handler(event, context):
    """
    Lambda 함수의 메인 핸들러
    Amazon Bedrock 에이전트에서 호출될 때 실행됩니다.
    """
    # 에이전트에서 전달된 정보 파싱
    actionGroup = event.get('actionGroup', '')  # 액션 그룹 이름
    function = event.get('function', '')  # 호출할 함수 이름
    parameters = event.get('parameters', [])  # 함수 매개변수들

    # 함수별 로직 실행
    if function == 'get_booking_details':
        booking_id = get_named_parameter(event, "booking_id")
        if booking_id:
            response = str(get_booking_details(booking_id))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'booking_id 매개변수가 누락되었습니다.'}}

    elif function == 'create_booking':
        date = get_named_parameter(event, "date")
        name = get_named_parameter(event, "name")
        hour = get_named_parameter(event, "hour")
        num_guests = get_named_parameter(event, "num_guests")

        if date and hour and num_guests:
            response = str(create_booking(date, name, hour, num_guests))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': '필수 매개변수가 누락되었습니다.'}}

    elif function == 'delete_booking':
        booking_id = get_named_parameter(event, "booking_id")
        if booking_id:
            response = str(delete_booking(booking_id))
            responseBody = {'TEXT': {'body': json.dumps(response)}}
        else:
            responseBody = {'TEXT': {'body': 'booking_id 매개변수가 누락되었습니다.'}}

    else:
        responseBody = {'TEXT': {'body': '잘못된 함수 호출입니다.'}}

    # 에이전트에게 반환할 응답 구성
    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }
    }

    function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
    print("Lambda 응답:", function_response)

    return function_response

#### 필요한 권한 생성

**Lambda 함수 권한 설정**
이제 Lambda 역할과 필요한 정책들을 생성해보겠습니다. Lambda 함수가 DynamoDB에 접근할 수 있도록 DynamoDB 정책을 생성하고 Lambda에 연결해야 합니다. 이를 위해 `create_lambda_role` 지원 함수를 사용하겠습니다.

In [ ]:
print("Lambda 함수를 위한 IAM 역할 생성 중...")
lambda_iam_role = create_lambda_role(agent_name, table_name)
print("Lambda IAM 역할이 성공적으로 생성되었습니다!")

#### 함수 생성하기

**Lambda 함수 배포**
이제 Lambda 함수 코드와 실행 역할이 준비되었으므로, 이를 Zip 파일로 패키징하고 Lambda 리소스를 생성하겠습니다.

In [ ]:
lambda_function_name = f'{agent_name}-lambda'  # Lambda 함수 이름
print(f"Lambda 함수 이름: {lambda_function_name}")

In [ ]:
print("Lambda 함수를 생성하고 배포하는 중...")
lambda_function = create_lambda(lambda_function_name, lambda_iam_role)
print("Lambda 함수가 성공적으로 생성되었습니다!")

### 4.3 에이전트에 필요한 IAM 정책 생성

**에이전트 권한 설정**
이제 지식 베이스, DynamoDB 테이블, Lambda 함수를 모두 생성했으므로 에이전트 자체를 만들어보겠습니다.

먼저 에이전트가 Bedrock 모델을 호출하고 지식 베이스를 쿼리할 수 있도록 하는 정책들과 IAM 역할을 생성해야 합니다. 이 에이전트가 선택한 AI 모델을 사용할 수 있도록 권한을 부여하겠습니다. `create_agent_role_and_policies` 함수를 사용하여 에이전트 역할과 필수 정책들을 생성합니다.

In [ ]:
print("에이전트 IAM 역할과 정책들을 생성하는 중...")
agent_role = create_agent_role_and_policies(agent_name, inference_profile, foundation_model, kb_id=kb_id)
print("에이전트 IAM 설정이 완료되었습니다!")

In [ ]:
agent_role

### 4.4 에이전트 생성하기

**드디어 에이전트 생성!**
필요한 IAM 역할이 생성되었으므로, 이제 bedrock agent 클라이언트를 사용하여 새로운 에이전트를 생성할 수 있습니다. 

이를 위해 boto3의 [`create_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent.html) API를 사용합니다. 에이전트 이름, 기본 AI 모델, 지시사항이 필요하며, 에이전트 설명도 제공할 수 있습니다. 

참고로 생성된 에이전트는 아직 준비되지 않은 상태입니다. 나중에 에이전트를 준비하고 액션을 호출하거나 다른 API를 사용하는 것에 집중하겠습니다.

In [ ]:
# 에이전트 역할 생성을 위한 대기 시간
print("에이전트 역할이 완전히 준비될 때까지 30초 대기...")
time.sleep(30)

print("Amazon Bedrock 에이전트를 생성하는 중...")
response = bedrock_agent_client.create_agent(
    agentName=agent_name,  # 에이전트 이름
    agentResourceRoleArn=agent_role['Role']['Arn'],  # IAM 역할 ARN
    description=agent_description,  # 에이전트 설명
    idleSessionTTLInSeconds=1800,  # 세션 유지 시간 (30분)
    foundationModel=inference_profile,  # 사용할 AI 모델
    instruction=agent_instruction,  # 에이전트 지시사항
)
print("에이전트가 성공적으로 생성되었습니다!")
response

**에이전트 ID 가져오기**
에이전트 ID를 저장해둡시다. 에이전트와 관련된 작업을 수행할 때 중요하게 사용됩니다.

In [ ]:
agent_id = response['agent']['agentId']
print(f"에이전트 ID: {agent_id}")

### 4.5 에이전트 액션 그룹 생성하기

**에이전트가 할 수 있는 작업들 정의하기**
이제 이전에 생성한 Lambda 함수를 사용하는 에이전트 액션 그룹을 생성하겠습니다. 

[`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) 함수가 이 기능을 제공합니다. 아직 에이전트 버전이나 별칭을 생성하지 않았으므로 `DRAFT` 버전을 사용합니다. 

에이전트에게 액션 그룹의 기능을 알려주기 위해, 액션 그룹이 수행할 수 있는 기능들을 설명하는 액션 그룹 설명을 제공합니다.

이 예제에서는 [`functionSchema`](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-function.html)를 사용하여 액션 그룹 기능을 제공합니다.

함수 스키마를 사용하여 함수를 정의하려면 각 함수의 `name`(이름), `description`(설명), `parameters`(매개변수)를 제공해야 합니다.

In [ ]:
# 에이전트가 수행할 수 있는 함수들 정의
agent_functions = [
    {
        'name': 'get_booking_details',
        'description': '레스토랑 예약 정보를 조회합니다',
        'parameters': {
            "booking_id": {
                "description": "조회할 예약 ID",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'create_booking',
        'description': '새로운 레스토랑 예약을 생성합니다',
        'parameters': {
            "date": {
                "description": "예약 날짜",
                "required": True,
                "type": "string"
            },
            "name": {
                "description": "예약자 이름",
                "required": True,
                "type": "string"
            },
            "hour": {
                "description": "예약 시간",
                "required": True,
                "type": "string"
            },
            "num_guests": {
                "description": "예약 인원 수",
                "required": True,
                "type": "integer"
            }
        }
    },
    {
        'name': 'delete_booking',
        'description': '기존 레스토랑 예약을 삭제합니다',
        'parameters': {
            "booking_id": {
                "description": "삭제할 예약 ID",
                "required": True,
                "type": "string"
            }
        }
    },
]

print("에이전트 함수 스키마 정의 완료:")
for func in agent_functions:
    print(f"  • {func['name']}: {func['description']}")

**함수 스키마로 액션 그룹 생성**
이제 함수 스키마를 사용하여 [`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) API로 에이전트 액션 그룹을 생성합니다.

In [ ]:
# 에이전트가 생성될 때까지 대기
print("에이전트 생성 완료를 위해 30초 대기...")
time.sleep(30)

# 에이전트 액션 그룹 생성
print("에이전트 액션 그룹을 구성하고 생성하는 중...")
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,  # 에이전트 ID
    agentVersion='DRAFT',  # 드래프트 버전 사용
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']  # Lambda 함수 ARN
    },
    actionGroupName=agent_action_group_name,  # 액션 그룹 이름
    functionSchema={
        'functions': agent_functions  # 함수 스키마
    },
    description=agent_action_group_description  # 액션 그룹 설명
)
print("에이전트 액션 그룹이 성공적으로 생성되었습니다!")

In [ ]:
agent_action_group_response

### 4.6 에이전트가 액션 그룹 Lambda를 호출할 수 있도록 허용

**Lambda 호출 권한 설정**
액션 그룹을 사용하기 전에, 에이전트가 액션 그룹과 연결된 Lambda 함수를 호출할 수 있도록 허용해야 합니다. 이는 [리소스 기반 정책](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-permissions.html#agents-permissions-lambda)을 통해 수행됩니다. 생성된 Lambda 함수에 리소스 기반 정책을 추가해보겠습니다.

In [ ]:
# Lambda 함수에 Bedrock 호출 권한 추가
print("Lambda 함수에 Bedrock 호출 권한을 추가하는 중...")
lambda_client = boto3.client('lambda')
response = lambda_client.add_permission(
    FunctionName=lambda_function_name,  # Lambda 함수 이름
    StatementId='allow_bedrock',  # 정책 문 ID
    Action='lambda:InvokeFunction',  # 허용할 액션
    Principal='bedrock.amazonaws.com',  # 호출 주체 (Bedrock 서비스)
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",  # 소스 ARN (우리 에이전트)
)
print("Lambda 호출 권한이 성공적으로 설정되었습니다!")

In [ ]:
response

### 4.7 지식 베이스를 에이전트에 연결하기

**에이전트와 지식 베이스 연결**
이제 에이전트를 생성했으므로 앞서 만든 지식 베이스를 연결할 수 있습니다.

In [ ]:
print("지식 베이스를 에이전트에 연결하는 중...")
response = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,  # 에이전트 ID
    agentVersion='DRAFT',  # 드래프트 버전
    description='고객이 메뉴의 요리에 대해 질문할 때 지식 베이스에 접근합니다.',  # 연결 설명
    knowledgeBaseId=kb_id,  # 지식 베이스 ID
    knowledgeBaseState='ENABLED'  # 지식 베이스 활성화
)
print("지식 베이스와 에이전트가 성공적으로 연결되었습니다!")

In [ ]:
response

### 4.8 에이전트 준비 및 별칭 생성

**에이전트 최종 준비하기**
내부 테스트에 사용할 수 있는 에이전트의 DRAFT 버전을 생성해보겠습니다.

In [ ]:
print("에이전트를 사용 가능 상태로 준비하는 중...")
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print("에이전트 준비 완료!")
print(response)

# 에이전트가 완전히 준비될 때까지 대기
print("에이전트 준비 완료를 위해 30초 대기...")
time.sleep(30)

**에이전트 별칭 생성**

테스트 별칭 ID `TSTALIASID`를 사용하여 에이전트의 DRAFT 버전을 호출하거나, 에이전트를 위한 새로운 별칭과 새로운 버전을 생성할 수 있습니다. 여기서는 나중에 생성된 별칭 ID로 호출할 수 있도록 에이전트 별칭을 생성하겠습니다.

In [ ]:
print("에이전트 별칭을 생성하는 중...")
response = bedrock_agent_client.create_agent_alias(
    agentAliasName='TestAlias',  # 별칭 이름
    agentId=agent_id,  # 에이전트 ID
    description='테스트용 별칭',  # 별칭 설명
)

alias_id = response["agentAlias"]["agentAliasId"]
print(f"에이전트 별칭이 생성되었습니다: {alias_id}")

print("별칭 생성 완료를 위해 30초 대기...")
time.sleep(30)

## 5. 에이전트 테스트하기

**드디어 에이전트 테스트 시간!**
이제 에이전트를 생성했으므로 `bedrock-agent-runtime` 클라이언트를 사용하여 이 에이전트를 호출하고 몇 가지 작업을 수행해보겠습니다. [`invoke_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/invoke_agent.html) API를 사용하여 에이전트를 호출할 수 있습니다.

In [ ]:
%%time
import json
from datetime import datetime

class DateTimeEncoder(json.JSONEncoder):
    """날짜/시간 객체를 JSON으로 직렬화하기 위한 커스텀 인코더"""
    def default(self, obj):
        if isinstance(obj, datetime):
            return obj.isoformat()
        return super().default(obj)


def invokeAgent(query, session_id, enable_trace=False, session_state=dict()):
    """
    에이전트를 호출하는 헬퍼 함수
    
    Args:
        query: 에이전트에게 보낼 질문/요청
        session_id: 세션 ID (대화 구분용)
        enable_trace: 디버그 추적 정보 표시 여부
        session_state: 세션 상태 (프롬프트 속성 등)
    
    Returns:
        에이전트의 응답
    """
    end_session: bool = False
    
    print(f"에이전트에게 질문: {query}")
    
    # 에이전트 API 호출
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,  # 사용자 입력
        agentId=agent_id,  # 에이전트 ID
        agentAliasId=alias_id,  # 에이전트 별칭 ID
        sessionId=session_id,  # 세션 ID
        enableTrace=enable_trace,  # 추적 활성화 여부
        endSession=end_session,  # 세션 종료 여부
        sessionState=session_state  # 세션 상태
    )
    
    if enable_trace:
        logger.info(pprint.pprint(agentResponse))
    
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                if enable_trace:
                    logger.info(f"최종 답변 ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                return agent_answer
                # 요청이 성공적으로 완료되었음을 나타내는 종료 이벤트
            elif 'trace' in event:
                if enable_trace:
                    logger.info(json.dumps(event['trace'], indent=2, cls=DateTimeEncoder))
            else:
                raise Exception("예상치 못한 이벤트가 발생했습니다.", event)
    except Exception as e:
        raise Exception("예상치 못한 이벤트가 발생했습니다.", e)

##### 지식 베이스 쿼리를 위한 에이전트 호출

**지식 베이스 테스트**
이제 지원 함수 `invokeAgent`를 사용하여 에이전트로 지식 베이스를 쿼리해보겠습니다.

In [ ]:
%%time
import uuid

print("테스트 1: 지식 베이스에서 메뉴 정보 조회")
session_id: str = str(uuid.uuid1())  # 고유한 세션 ID 생성
query = "어린이 메뉴에는 어떤 전채요리가 있나요?"

response = invokeAgent(query, session_id)
print("\n에이전트 응답:")
print("=" * 60)
print(response)
print("=" * 60)

##### 액션 그룹 함수 실행을 위한 에이전트 호출

**예약 기능 테스트**
이제 액션 그룹 기능을 테스트해보고 새로운 예약을 생성해보겠습니다.

In [ ]:
%%time
print("테스트 2: 새 예약 생성하기")
query = "안녕하세요, 저는 안나입니다. 5월 5일 저녁 8시에 2명 예약을 하고 싶습니다."

response = invokeAgent(query, session_id)
print("\n에이전트 응답:")
print("=" * 60)  
print(response)
print("=" * 60)

##### 프롬프트 속성을 사용한 에이전트 호출

**잘했습니다!** 에이전트를 사용하여 첫 번째 예약을 만들었습니다. 하지만 레스토랑에서 테이블을 예약할 때는 종종 우리 이름을 알고 있는 시스템에 이미 로그인되어 있습니다. 에이전트도 이를 알고 있다면 얼마나 좋을까요?

**세션 컨텍스트 활용**
이를 위해 세션 컨텍스트를 사용하여 프롬프트에 일부 속성을 제공할 수 있습니다. 이 경우 [`promptSessionAttributes`](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-session-state.html) 매개변수를 사용하여 프롬프트에 직접 제공하겠습니다. 또한 에이전트가 우리 이름을 기억하지 않도록 새로운 세션 ID를 시작하겠습니다.

In [ ]:
%%time
print("테스트 3: 세션 속성을 사용한 예약 생성")
session_id: str = str(uuid.uuid1())  # 새로운 세션 ID
query = "5월 5일 저녁 8시에 2명 예약을 하고 싶습니다."

# 세션 상태에 사용자 이름 제공
session_state = {
    "promptSessionAttributes": {
        "name": "김철수"  # 미리 알고 있는 사용자 이름
    }
}

response = invokeAgent(query, session_id, session_state=session_state)
print("\n에이전트 응답:")
print("=" * 60)
print(response)
print("=" * 60)

##### 프롬프트 속성 검증

**예약 이름 확인**
이제 세션 컨텍스트를 사용하여 예약이 올바른 이름으로 이루어졌는지 검증해보겠습니다.

In [ ]:
%%time
print("테스트 4: 마지막 예약의 이름 확인")
query = "방금 만든 예약에서 사용된 이름이 무엇인가요?"

response = invokeAgent(query, session_id)
print("\n에이전트 응답:")
print("=" * 60)
print(response)
print("=" * 60)

##### 새 세션에서 데이터베이스 정보 조회

**예약 시스템 검증**  
다음으로, 예약 시스템이 제대로 작동하는지 확인해보겠습니다. 이전 예약 ID를 사용하여 새로운 세션 ID로 예약 세부 정보를 조회해보겠습니다.

**중요:** 다음 쿼리에서는 생성된 예약 ID로 정보를 교체해야 합니다.

In [ ]:
%%time
print("테스트 5: 특정 예약 ID로 정보 조회")
session_id: str = str(uuid.uuid1())  # 완전히 새로운 세션
query = "예약 ID 65d86f1a에 대한 정보를 알려주세요"  # 실제 생성된 예약 ID로 변경하세요

response = invokeAgent(query, session_id)
print("\n에이전트 응답:")
print("=" * 60)
print(response)
print("=" * 60)

##### 예약 취소

**예약 변경**
계획이 바뀌었으므로 에이전트를 사용하여 방금 만든 예약을 취소해보겠습니다.

In [ ]:
%%time
print("테스트 6: 예약 삭제하기")
query = "예약 ID 65d86f1a를 삭제하고 싶습니다"  # 실제 생성된 예약 ID로 변경하세요

response = invokeAgent(query, session_id)
print("\n에이전트 응답:")
print("=" * 60)
print(response)
print("=" * 60)

**삭제 확인**
모든 것이 제대로 작동했는지 확인해보겠습니다.

In [ ]:
%%time
print("테스트 7: 삭제된 예약 조회 시도")
session_id: str = str(uuid.uuid1())  # 새로운 세션
query = "예약 ID 65d86f1a에 대한 정보를 알려주세요"  # 삭제한 예약 ID로 변경하세요

response = invokeAgent(query, session_id)
print("\n에이전트 응답:")
print("=" * 60)
print(response)
print("=" * 60)

##### 프롬프트 속성을 사용한 상황 인식 처리

**실제 애플리케이션에서의 상황 인식**
실제 애플리케이션에서는 맥락이 정말 중요합니다. 현재 날짜와 주변 날짜를 고려하여 예약을 하고 싶습니다. Amazon Bedrock 에이전트는 프롬프트 속성을 통해 에이전트에게 시간적 맥락을 제공할 수도 있습니다. 내일 예약으로 테스트해보겠습니다.

In [ ]:
%%time
print("테스트 8: 날짜 맥락을 사용한 예약 생성")
# 내일 테이블 예약하기
session_id: str = str(uuid.uuid1())
query = "내일 저녁 8시에 2명 예약을 하고 싶습니다."

# 현재 날짜 정보를 세션 속성에 추가
session_state = {
    "promptSessionAttributes": {
        "name": "김철수",
        "오늘 날짜": datetime.now().strftime("%Y-%m-%d, %A")  # 현재 날짜와 요일
    }
}

response = invokeAgent(query, session_id, session_state=session_state)
print("\n에이전트 응답:")
print("=" * 60)
print(response)
print("=" * 60)

**예약 확인**
마지막으로 우리의 예약을 검증해보겠습니다.

**중요:** 새로 생성된 예약 ID로 교체해주세요.

In [ ]:
%%time
print("테스트 9: 새로 생성된 예약 확인")
session_id: str = str(uuid.uuid1())
query = "예약 ID c081638f에 대한 정보를 알려주세요"  # 새로 생성된 예약 ID로 변경하세요

response = invokeAgent(query, session_id)
print("\n에이전트 응답:")
print("=" * 60)
print(response)
print("=" * 60)

##### 추적 기능이 활성화된 에이전트 호출

**에이전트 내부 동작 살펴보기**
Amazon Bedrock 에이전트는 [추적(Trace)](https://docs.aws.amazon.com/bedrock/latest/userguide/trace-events.html) 기능을 사용하여 에이전트가 조율하는 단계의 세부 정보를 제공합니다. 에이전트 호출 중에 추적을 활성화할 수 있습니다. 이제 추적이 활성화된 상태로 에이전트를 호출해보겠습니다.

In [ ]:
%%time
print("테스트 10: 추적 기능으로 에이전트 내부 동작 확인")
session_id: str = str(uuid.uuid1())
query = "성인 메뉴에는 어떤 디저트가 있나요?"

# enable_trace=True로 설정하여 에이전트의 내부 동작 과정을 볼 수 있습니다
response = invokeAgent(query, session_id, enable_trace=True)
print("\n최종 에이전트 응답:")
print("=" * 60)
print(response)
print("=" * 60)

## 에이전트 평가 프레임워크 - 에이전트 테스트 (선택사항)

**체계적인 에이전트 성능 평가**
[에이전트 평가 프레임워크](https://awslabs.github.io/agent-evaluation/)는 Bedrock 에이전트의 성능, 정확성, 효과성을 평가하기 위한 체계적인 접근 방식을 제공합니다.

다음 단계들은 선택사항이며, 테스트 케이스를 작성하고 Bedrock 에이전트에 대해 실행하는 방법을 보여줍니다.

In [ ]:
print("에이전트 평가 프레임워크 설치 중...")
!python3 -m pip install agent-evaluation==0.2.0  # 에이전트 평가 프레임워크 도구 설치

print("에이전트 평가 프레임워크 설치 확인 중...")
!which agenteval  # 에이전트 평가 프레임워크가 제대로 설치되었는지 확인
print("설치 완료!")

**평가 설정 준비**
- 다음 섹션들은 이 노트북으로 생성된 에이전트 ID와 별칭 ID를 5번 줄에 제공하여 `agenteval.yml` 파일을 준비합니다. `agenteval.yml` 파일에서는 에이전트를 테스트하기 위해 정의된 다양한 테스트 케이스를 찾을 수 있습니다.
- 작성 시점에서는 평가자로 Claude 3 Sonnet만 지원됩니다. yml 파일에 지정된 Claude-3 모델은 Claude 3 Sonnet을 의미합니다.

In [ ]:
# agent_id와 alias_id가 이미 정의되어 있다고 가정
print(f"에이전트 ID: {agent_id}")  # 참조용 에이전트 ID 출력
print(f"별칭 ID: {alias_id}")     # 참조용 별칭 ID 출력

print("agenteval.yml 파일 업데이트 중...")
# 먼저 YAML 파일 읽기
with open('agenteval.yml', 'r') as file:
    content = file.read()
    
# 값들 교체
updated_content = content.replace('[agent_id]', agent_id)
updated_content = updated_content.replace('None', alias_id)

# 파일에 다시 쓰기
with open('agenteval.yml', 'w') as file:
    file.write(updated_content)
    
print("agenteval.yml 파일이 업데이트되었습니다!")

In [ ]:
# 저장소의 일부인 정의된 테스트 케이스들을 실행합니다 (즉, agenteval.yml 파일)
print("에이전트 평가 테스트를 실행합니다...")
!agenteval run
print("평가 완료!")

**평가 결과 확인**
- 위의 출력은 테스트 평가 결과를 보여줍니다. 다음 생성된 파일에서 테스트 평가에 대한 상세한 보고서를 찾을 수 있습니다: `agenteval_summary.md` 우클릭하고 "다음으로 열기 -> 마크다운 미리보기"를 선택하여 미리 볼 수 있습니다.

- 테스트가 실행되면 몇 가지 새로운 파일들도 생성된 것을 알 수 있습니다 (예: `check_number_of_vacation.json`). 여기서 테스트 사용자와 테스트 에이전트 간의 테스트 대화에서 자세한 추적 정보를 찾을 수 있습니다.

## 6. 리소스 정리하기

**비용 절약을 위한 정리 작업**
불필요한 비용을 피하기 위해 생성된 모든 관련 리소스를 삭제하겠습니다.

In [ ]:
print("AWS 리소스들을 정리하는 중...")
clean_up_resources(
    table_name, lambda_function, lambda_function_name, agent_action_group_response, agent_functions, 
    agent_id, kb_id, alias_id
)
print("주요 리소스 정리가 완료되었습니다!")

In [ ]:
# 에이전트 역할과 정책들 삭제
print("에이전트 IAM 역할과 정책들을 삭제하는 중...")
delete_agent_roles_and_policies(agent_name)
print("IAM 리소스 정리가 완료되었습니다!")

In [ ]:
# 지식 베이스 삭제
print("지식 베이스와 관련 리소스들을 삭제하는 중...")
knowledge_base.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)
print("지식 베이스 정리가 완료되었습니다!")
print("\n모든 리소스 정리가 성공적으로 완료되었습니다!")